In [51]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import folium
import geopandas as gpd
from shapely.geometry import LineString, box, mapping, Point
from shapely import wkt
import rasterio
from PIL import ImageDraw, Image
from rasterio.mask import mask
from rasterio.merge import merge

orth_tiff_folder_2023 = '/data/shared/rsdata/orthophotos/TEAK/NEON_images-camera-ortho-mosaic/NEON.D17.TEAK.DP3.30010.001.2023-07.basic.20240905T014815Z.PROVISIONAL'
# shapefile_path = "/data/shared/src/arojas/NEON/data/raw/spatial/baseplots/TEAK_baseplots_utm/TEAK_baseplots_utm.shp"

shapefile_path_original = "/data/shared/src/arojas/NEON/data/raw/spatial/baseplots/TEAK_baseplots_utm/TEAK_baseplots_utm.shp"
shapefile_path = '/data/shared/src/STV/NEON_TEAK/allen/data/shapefiles/TEAK_shapefile_2m_buffer/TEAK_shapefile_2m_buffer.shp'
shapefile_path1 = '/data/shared/src/STV/NEON_TEAK/allen/data/shapefiles/TEAK_shapefile_no_buffer/TEAK_shapefile_no_buffer.shp'
shapefile_path2 = '/data/shared/src/STV/NEON_TEAK/allen/data/shapefiles/TEAK_shapefile_no_buffer_resized/TEAK_shapefile_no_buffer_resized.shp'
root_output_path = '/data/shared/src/STV/NEON_TEAK/allen/figs/veg_input'
date_subfolder = '2024_09_30'
date_output_path = os.path.join(root_output_path, date_subfolder)

tif_output_path = '/data/shared/src/STV/NEON_TEAK/allen/data/clipped_tif/TEAK_shapefile_2m_buffer'
tree_loc_path = '/data/shared/src/STV/NEON_TEAK/allen/data/tree_locations/TEAK_shapefile_2m_buffer'

app_indv_100_locations_df = pd.read_csv(os.path.join(date_output_path, "TEAK_entries_perplot_location_only_100.csv"))
app_indv_400_locations_df = pd.read_csv(os.path.join(date_output_path, "TEAK_entries_perplot_location_only_400.csv"))

In [52]:
def get_date_subfolder():
    current_date = datetime.now()
    return current_date.strftime('%Y_%m_%d')

def check_crs(tif_directory):
    ### Check if all TIFF files in the directory have the same CRS.  Returns the crs if this is true

    common_crs = None
    all_files_checked = True

    # Loop through all .tif files in the directory
    for filename in os.listdir(tif_directory):
        if filename.endswith(".tif") or filename.endswith(".tiff"):
            file_path = os.path.join(tif_directory, filename)
            
            # Open the TIFF file and get its CRS
            with rasterio.open(file_path) as src:
                crs = src.crs
                
                if common_crs is None:
                    # Set the first CRS as the common CRS to check against
                    common_crs = crs
                else:
                    # Compare the current file's CRS to the common CRS
                    if crs != common_crs:
                        all_files_checked = False
                        raise ValueError(f"CRS mismatch in {filename}: {crs} differs from {common_crs}")
    
    # If no mismatches are found, print success message
    if all_files_checked:
        print(f"All TIFF files have the same CRS: {common_crs}")

    return common_crs

def popup_construct(each_point):
    lat = round(each_point['latitude'], 4)
    long = round(each_point['longitude'], 4)
    pop = (
        'plotID: ' + str(each_point['plotID_x']) + ',<br><br>subplotID: ' + str(each_point['subplotID']) + 
        ',<br><br>individualID: ' + str(each_point['individualID']) + 
        ',<br><br>lat: ' + str(lat) + ', ' + '<br><br>long: ' + str(long)
    )
    return pop

def add_circles(m, df, color):    
    # Determine the layer group name based on the color
    layer_group_name = f"Tree Circles_100m2" if color == 'red' else f"Tree Circles_400m2"
    # Create a FeatureGroup for the circles
    circles_layer_group = folium.FeatureGroup(name=layer_group_name)

    for index, row in df.iterrows():
        iframe = folium.IFrame(popup_construct(row))
        popup = folium.Popup(iframe, min_width=300, max_width=300)
        
        if pd.notna(row['easting']):
            
            folium.Circle(
                location=[row['latitude'], row['longitude']],
                radius=1,
                fill=True,
                popup=popup,
                color=color,
                name = row['individualID']
            ).add_to(circles_layer_group)
    circles_layer_group.add_to(m)  # Add the layer group to the map
    return m

def add_markers(m, df, color):    
    # Determine the layer group name based on the color
    layer_group_name = f"Tree Markers_100m2" if color == 'red' else f"Tree Markers_400m2"

    # Create a FeatureGroup for the markers
    markers_layer_group = folium.FeatureGroup(name=layer_group_name)

    for index, row in df.iterrows():
        iframe = folium.IFrame(popup_construct(row))
        popup = folium.Popup(iframe, min_width=300, max_width=300)
        
        if pd.notna(row['easting']):
            folium.Marker(
                location=[row['latitude'], row['longitude']],
                popup=popup,
                icon=folium.Icon(color=color),
                name = (row['individualID'] + "marker")
            ).add_to(markers_layer_group)
    markers_layer_group.add_to(m)  # Add the layer group to the map
    return m

def add_lines_to_polygons(gdf):
    lines = []
    for _, row in gdf.iterrows():
        bounds = row.geometry.bounds  # (minx, miny, maxx, maxy)
        minx, miny, maxx, maxy = bounds
        
        # Calculate the positions of the lines that will subdivide the square
        midx1 = minx + (maxx - minx) / 4
        midx2 = minx + (maxx - minx) / 2
        midx3 = minx + 3 * (maxx - minx) / 4
        midy1 = miny + (maxy - miny) / 4
        midy2 = miny + (maxy - miny) / 2
        midy3 = miny + 3 * (maxy - miny) / 4
        
        # Create vertical lines
        vertical_line1 = LineString([(midx1, miny), (midx1, maxy)])
        vertical_line2 = LineString([(midx2, miny), (midx2, maxy)])
        vertical_line3 = LineString([(midx3, miny), (midx3, maxy)])
        
        # Create horizontal lines
        horizontal_line1 = LineString([(minx, midy1), (maxx, midy1)])
        horizontal_line2 = LineString([(minx, midy2), (maxx, midy2)])
        horizontal_line3 = LineString([(minx, midy3), (maxx, midy3)])
        
        # Append lines to the list
        lines.append(vertical_line1)
        lines.append(vertical_line2)
        lines.append(vertical_line3)
        lines.append(horizontal_line1)
        lines.append(horizontal_line2)
        lines.append(horizontal_line3)
    
    # Convert the list of lines into a GeoDataFrame
    lines_gdf = gpd.GeoDataFrame(geometry=lines, crs=gdf.crs)
    return lines_gdf

# Function to plot TIFF file boundaries
def plot_tiff_boundaries(m, tiff_dir):
    # Create a FeatureGroup for TIFF boundaries
    tiff_layer_group = folium.FeatureGroup(name="TIFF Boundaries")

    for tiff_file in os.listdir(tiff_dir):
        if tiff_file.endswith('.tif'):
            tiff_path = os.path.join(tiff_dir, tiff_file)
            with rasterio.open(tiff_path) as src:
                bounds = src.bounds
                gdf_tiff = gpd.GeoDataFrame(index=[0], crs=src.crs, geometry=[box(*bounds)])
                gdf_tiff = gdf_tiff.to_crs(epsg=4326)
                
                # Add the GeoJson for this TIFF boundary to the FeatureGroup
                folium.GeoJson(
                    gdf_tiff,
                    name=tiff_file
                ).add_to(tiff_layer_group)
    
    # Add the FeatureGroup to the map
    tiff_layer_group.add_to(m)

def filter_polygons_with_points(df_to_export, merged_100_df, merged_400_df):
    # Convert DataFrames to GeoDataFrames
    points_100_gdf = gpd.GeoDataFrame(merged_100_df, geometry=gpd.points_from_xy(merged_100_df.longitude, merged_100_df.latitude), crs='EPSG:4326')
    points_400_gdf = gpd.GeoDataFrame(merged_400_df, geometry=gpd.points_from_xy(merged_400_df.longitude, merged_400_df.latitude), crs='EPSG:4326')

    # Convert df_to_export back to GeoDataFrame
    df_to_export_gdf = gpd.GeoDataFrame(df_to_export.copy(), geometry=df_to_export['geometry_wkt'].apply(wkt.loads), crs="EPSG:4326")
    
    # Initialize list to store plotIDs of polygons that contain points
    valid_plotIDs = set()

    # Check each polygon in the shapefile
    for _, polygon_row in df_to_export_gdf.iterrows():
        polygon_geom = polygon_row.geometry
        plot_id = polygon_row['plotID']

        # Debug: Output the plotID being checked
        print(f"Checking polygon with plotID: {plot_id}")

        # Check if there are any points in merged_100_df within this polygon
        points_within_polygon_100 = points_100_gdf[points_100_gdf.geometry.within(polygon_geom)]
        points_within_polygon_400 = points_400_gdf[points_400_gdf.geometry.within(polygon_geom)]

        if not points_within_polygon_100.empty or not points_within_polygon_400.empty:
            print(f"Polygon {plot_id} contains {len(points_within_polygon_100) + len(points_within_polygon_400)} points.\n")
            valid_plotIDs.add(plot_id)
        else:
            print(f"Polygon {plot_id} does not contain any points.\n")

    # Filter gdf to keep only polygons that have points within them
    df_to_export_filtered = df_to_export_gdf[df_to_export_gdf['plotID'].isin(valid_plotIDs)]

    # Debug: Output the length of the resulting DataFrame
    print(f"Number of rows in original df_to_export: {len(df_to_export)}")
    print(f"Number of rows removed: {len(df_to_export) - len(df_to_export_filtered)}")
    print(f"Number of rows in filtered df_to_export: {len(df_to_export_filtered)}")

    return df_to_export_filtered

def add_filtered_polygons_to_map(m, gdf_filtered, crs):
    ### Add filtered polygons to a Folium map with clickable popups.
    # Function to construct the popup content
    
    def popup_construct(polygon):
        plot_id = polygon['plotID']
        plot_type = polygon['plotType']
        plot_dim = polygon['plotDim']
        return (
            f'plotID: {plot_id}<br>'
            f'plotType: {plot_type}<br>'
            f'plotDim: {plot_dim}'
        )
    
    # Style function to set the opacity of the polygons
    def style_function(feature):
        return {
            'fillOpacity': 0.3,
            'color': 'blue',   # Outline color
            'weight': 2        # Outline thickness
        }
    
    # Create a FeatureGroup for polygons
    polygons_layer_group = folium.FeatureGroup(name="Filtered Polygons")

    # Iterate through each row in the filtered GeoDataFrame
    for _, row in gdf_filtered.iterrows():
        # Create a GeoJson feature with a popup and add it to the FeatureGroup
        folium.GeoJson(
            row['geometry'],
            popup=folium.Popup(popup_construct(row), max_width=300),
            style_function=style_function,
            name=f"Polygon {row['plotID']}"  # Name the layer using plotID or any other identifier
        ).add_to(polygons_layer_group)
    
    # Add the FeatureGroup to the map
    polygons_layer_group.add_to(m)

def create_folium_map(merged_100_df, merged_400_df, root_path, shapefile_path, tiff_folder, crs, map_location=[37.00583, -119.00602], zoom_start=12):
    m = folium.Map(location=map_location, zoom_start=zoom_start)
    
    # Add circles for the 100-meter dataset
    add_circles(m, merged_100_df, 'red')
    add_circles(m, merged_400_df, 'blue')
    
    # Add markers for the 400-meter dataset
    add_markers(m, merged_100_df, 'red')
    add_markers(m, merged_400_df, 'blue')

    jpeg_csv_path = os.path.join(tiff_folder, "clipped_tif")
    jpeg_csv_file = jpeg_csv_path + "/clipped_jpegs_info.csv"

    gdf = gpd.read_file(shapefile_path)
    gdf = gdf.to_crs(epsg=4326)
    print(gdf.columns)
    print(gdf.head())

    # # Add the shapefile to the map
    # folium.GeoJson(gdf).add_to(m)

    plot_tiff_boundaries(m, tiff_folder)

    # Convert the geometry to WKT format
    gdf['geometry_wkt'] = gdf['geometry'].apply(lambda geom: geom.wkt)

    # Select only the columns you want to export
    df_to_export = gdf[['plotID', 'plotType', 'plotDim', 'geometry_wkt']]

    df_to_export_filtered = filter_polygons_with_points(df_to_export, merged_100_df, merged_400_df)
    # add_jpegs_to_map(m, jpeg_csv_file, df_to_export_filtered)

    add_filtered_polygons_to_map(m, df_to_export_filtered, crs)
    lines_gdf = add_lines_to_polygons(df_to_export_filtered)
    
    # Create and add lines to the map
    folium.GeoJson(lines_gdf, name="Subdivision Lines").add_to(m)

    # # Add layer control to toggle overlays
    folium.LayerControl().add_to(m)

    map_output_filename = root_path + '/TEAK_100_vs_400_new.html'
    csv_output_filename = root_path + '/shapefile_geometries.csv'

    # Export to CSV
    df_to_export_filtered.to_csv(csv_output_filename, index=False)
    m.save(map_output_filename)
    
    return m

def add_jpegs_to_map(folium_map, jpeg_csv_file, gdf):
    # Read the CSV file containing JPEG information
    jpeg_df = pd.read_csv(jpeg_csv_file)
    csv_dir = os.path.dirname(jpeg_csv_file)
    grouped = jpeg_df.groupby('polygon_id')

    # Create a FeatureGroup for JPEG layers
    jpeg_layer_group = folium.FeatureGroup(name="JPEG Images")

    for polygon_id, group in grouped:
        images = []
        geom = gdf[gdf['plotID'] == polygon_id].geometry.iloc[0]

        if geom is None:
            print(f"No geometry found for polygon_id: {polygon_id}")
            continue

        for _, row in group.iterrows():
            jpeg_filename = row['jpeg_filename']
            jpeg_path = os.path.join(csv_dir, jpeg_filename)
            if not os.path.exists(jpeg_path):
                print(f"JPEG file not found: {jpeg_path}")
                continue

            img = Image.open(jpeg_path)
            images.append(img)

        images = images[::-1]
        total_height = sum(img.height for img in images)
        max_width = max(img.width for img in images)
        combined_image = Image.new('RGB', (max_width, total_height))
        y_offset = 0
        for img in images:
            combined_image.paste(img, (0, y_offset))
            y_offset += img.height

        combined_image_path = os.path.join(csv_dir, f"combined_{polygon_id}.jpg")
        combined_image.save(combined_image_path)

        # Get the bounds of the polygon
        bounds = geom.bounds
        print(f"Adding image overlay for polygon_id: {polygon_id}")
        print(f"Image file path: {combined_image_path}")
        print(f"Bounds: [[{bounds[1]}, {bounds[0]}], [{bounds[3]}, {bounds[2]}]]")

        # Create an image overlay for the combined JPEG on the Folium map
        img_overlay = folium.raster_layers.ImageOverlay(
            name=f"Image_{polygon_id}",
            image=combined_image_path,
            bounds=[[bounds[1], bounds[0]], [bounds[3], bounds[2]]],
            opacity=1,
            interactive=True,
            cross_origin=False,
            zindex=1
        )

        img_overlay.add_to(jpeg_layer_group)

    # Add the JPEG layer group to the map
    jpeg_layer_group.add_to(folium_map)

In [53]:
tif_crs = check_crs(orth_tiff_folder_2023)

print(tif_crs)

All TIFF files have the same CRS: EPSG:32611
EPSG:32611


In [54]:
create_folium_map(app_indv_100_locations_df, app_indv_400_locations_df, date_output_path, shapefile_path_original, orth_tiff_folder_2023, tif_crs)

Index(['plotID', 'pointID', 'country', 'state', 'county', 'domain', 'domainID',
       'siteName', 'siteID', 'plotType', 'subtype', 'subSpec', 'plotSize',
       'plotDim', 'latitude', 'longitude', 'datum', 'utmZone', 'easting',
       'northing', 'horzUncert', 'elevation', 'vertUncert', 'minElev',
       'maxElev', 'slope', 'aspect', 'nlcdClass', 'soilOrder', 'coordSrc',
       'date', 'gpsLogs', 'plotPdop', 'plotHdop', 'appMods', 'plotEdge',
       'geometry'],
      dtype='object')
     plotID pointID       country state  county             domain domainID  \
0  TEAK_001      41  unitedStates    CA  Fresno  Pacific Southwest      D17   
1  TEAK_004      41  unitedStates    CA  Fresno  Pacific Southwest      D17   
2  TEAK_029      41  unitedStates    CA  Fresno  Pacific Southwest      D17   
3  TEAK_024      41  unitedStates    CA  Fresno  Pacific Southwest      D17   
4  TEAK_026      41  unitedStates    CA  Fresno  Pacific Southwest      D17   

               siteName siteID     

In [55]:
def check_polygon_within_tiff_bounds(shapefile_path, tiff_folder):
    ### Check if the bounds of polygons in a GeoDataFrame exist within any of the TIFF files in a folder.

    # Read the shapefile into a GeoDataFrame
    gdf = gpd.read_file(shapefile_path)
    
    # Prepare a dictionary to hold results
    result = {}
    
    # Iterate over each polygon in the GeoDataFrame
    for index, row in gdf.iterrows():
        polygon_geom = row['geometry']
        polygon_id = row['plotID']
        result[polygon_id] = []
        
        # Iterate over all TIFF files in the specified folder
        for tiff_file in os.listdir(tiff_folder):
            if tiff_file.endswith('.tif'):
                tiff_path = os.path.join(tiff_folder, tiff_file)

                with rasterio.open(tiff_path) as src:
                    
                    # Get the TIFF bounds in the CRS
                    minx, miny, maxx, maxy = src.bounds
                    
                    # Create the bounding box in the TIFF CRS
                    tiff_bounds = box(minx, miny, maxx, maxy)
                    
                    # Check if the polygon's geometry intersects with the TIFF's bounds
                    if polygon_geom.intersects(tiff_bounds):
                        result[polygon_id].append(tiff_file)
    
    return result

In [56]:
polygon_tiff_matches = check_polygon_within_tiff_bounds(shapefile_path, orth_tiff_folder_2023)

In [57]:
sorted_dict = dict(sorted(polygon_tiff_matches.items()))
# Print the result
for plot_id, tiffs in sorted_dict.items():
    print(f"Polygon {plot_id} is within the bounds of TIFF files: {', '.join(tiffs)}")

Polygon TEAK_001 is within the bounds of TIFF files: 2023_TEAK_6_320000_4094000_image.tif
Polygon TEAK_002 is within the bounds of TIFF files: 2023_TEAK_6_318000_4094000_image.tif
Polygon TEAK_003 is within the bounds of TIFF files: 2023_TEAK_6_321000_4098000_image.tif
Polygon TEAK_005 is within the bounds of TIFF files: 2023_TEAK_6_323000_4103000_image.tif
Polygon TEAK_006 is within the bounds of TIFF files: 2023_TEAK_6_316000_4094000_image.tif
Polygon TEAK_007 is within the bounds of TIFF files: 2023_TEAK_6_317000_4096000_image.tif
Polygon TEAK_010 is within the bounds of TIFF files: 2023_TEAK_6_317000_4098000_image.tif
Polygon TEAK_011 is within the bounds of TIFF files: 2023_TEAK_6_321000_4097000_image.tif
Polygon TEAK_012 is within the bounds of TIFF files: 2023_TEAK_6_321000_4098000_image.tif
Polygon TEAK_013 is within the bounds of TIFF files: 2023_TEAK_6_318000_4093000_image.tif, 2023_TEAK_6_318000_4094000_image.tif
Polygon TEAK_014 is within the bounds of TIFF files: 2023_TEAK

In [58]:
def clip_polygons_to_tiffs_and_merge(polygon_shapefile, polygon_tiff_matches, tiff_folder, output_folder):
    # Load the shapefile into a GeoDataFrame
    gdf = gpd.read_file(polygon_shapefile)
    
    # Assuming CRS of shapefile and TIFF files are the same
    tiff_crs = None
    for tiff_file in os.listdir(tiff_folder):
        if tiff_file.endswith('.tif'):
            with rasterio.open(os.path.join(tiff_folder, tiff_file)) as src:
                tiff_crs = src.crs
                break

    if not tiff_crs:
        raise ValueError("No TIFF files found or no CRS information available.")

    os.makedirs(output_folder, exist_ok=True)

    csv_data = []
    for polygon_id, tiff_files in polygon_tiff_matches.items():
        print(f"Processing polygon {polygon_id}...")
        polygon_geom = gdf[gdf['plotID'] == polygon_id].geometry.iloc[0]
        
        temp_clipped_tifs = []

        for tiff_file in tiff_files:
            tiff_path = os.path.join(tiff_folder, tiff_file)
            print(f"Clipping with TIFF file: {tiff_file}")
            
            with rasterio.open(tiff_path) as src:
                tiff_bounds = src.bounds
                if polygon_geom.bounds[0] > tiff_bounds.right or polygon_geom.bounds[2] < tiff_bounds.left or polygon_geom.bounds[1] > tiff_bounds.top or polygon_geom.bounds[3] < tiff_bounds.bottom:
                    print(f"Polygon {polygon_id} is out of bounds of TIFF file {tiff_file}.")
                    continue
                
                out_image, out_transform = mask(src, [mapping(polygon_geom)], crop=True)
                
                if out_image.shape[1] == 0 or out_image.shape[2] == 0:
                    print(f"Polygon {polygon_id} does not overlap with TIFF file {tiff_file}.")
                    continue
                
                # Save the clipped TIFF to a temporary file
                temp_tif_path = os.path.join(output_folder, f"temp_{polygon_id}_{os.path.basename(tiff_file)}")
                with rasterio.open(temp_tif_path, 'w', driver='GTiff', height=out_image.shape[1], width=out_image.shape[2], count=out_image.shape[0], dtype=out_image.dtype, crs=src.crs, transform=out_transform) as dst:
                    dst.write(out_image)

                temp_clipped_tifs.append(temp_tif_path)

        # If multiple TIFFs are clipped, merge them
        if len(temp_clipped_tifs) > 1:
            print(f"Merging TIFFs for polygon {polygon_id}...")
            src_files_to_mosaic = [rasterio.open(tif) for tif in temp_clipped_tifs]
            mosaic, out_transform = merge(src_files_to_mosaic)
            
            # Save the merged TIFF
            merged_tif_path = os.path.join(output_folder, f"{polygon_id}_merged.tif")
            with rasterio.open(merged_tif_path, 'w', driver='GTiff', height=mosaic.shape[1], width=mosaic.shape[2], count=mosaic.shape[0], dtype=mosaic.dtype, crs=tiff_crs, transform=out_transform) as dst:
                dst.write(mosaic)
                
            # Clean up temporary files
            for tif in temp_clipped_tifs:
                os.remove(tif)
            
            final_tif_path = merged_tif_path
        elif len(temp_clipped_tifs) == 1:
            final_tif_path = os.path.join(output_folder, f"{polygon_id}_clipped.tif")
            os.rename(temp_clipped_tifs[0], final_tif_path)
        else:
            final_tif_path = None
        
        # JPEG Image Processing
        if final_tif_path:
            clipped_image = Image.open(final_tif_path)

            # Convert TIFF to RGB for JPEG
            if clipped_image.mode != 'RGB':
                clipped_image = clipped_image.convert('RGB')

            # Define size for padded image
            clipped_width, clipped_height = clipped_image.size
            padded_width = max(clipped_width, 100)
            padded_height = max(clipped_height, 100)
            
            # Create a padded image
            padded_image = Image.new('RGB', (padded_width, padded_height), (0, 0, 0))
            paste_x = (padded_width - clipped_width) // 2
            paste_y = (padded_height - clipped_height) // 2
            padded_image.paste(clipped_image, (paste_x, paste_y))

            # Save JPEG
            jpeg_filename = f"{polygon_id}.jpg"
            jpeg_path = os.path.join(output_folder, jpeg_filename)
            padded_image.save(jpeg_path, 'JPEG')

            # Collect data for CSV
            csv_data.append({
                'polygon_id': polygon_id,
                'jpeg_filename': jpeg_filename,
                'tiff_file': os.path.basename(final_tif_path),
                'geometry': polygon_geom.wkt,
                'crs': tiff_crs.to_string()
            })

            print(f"Saved clipped raster and JPEG for polygon {polygon_id}")

    # Save CSV data to file
    csv_df = pd.DataFrame(csv_data)
    csv_df.to_csv(os.path.join(output_folder, 'clipped_jpegs_info.csv'), index=False)

    print("Clipping, merging, and JPEG creation completed. CSV file saved.")

In [59]:
clip_polygons_to_tiffs_and_merge(shapefile_path, polygon_tiff_matches, orth_tiff_folder_2023, tif_output_path)

Processing polygon TEAK_001...
Clipping with TIFF file: 2023_TEAK_6_320000_4094000_image.tif
Saved clipped raster and JPEG for polygon TEAK_001
Processing polygon TEAK_002...
Clipping with TIFF file: 2023_TEAK_6_318000_4094000_image.tif
Saved clipped raster and JPEG for polygon TEAK_002
Processing polygon TEAK_003...
Clipping with TIFF file: 2023_TEAK_6_321000_4098000_image.tif
Saved clipped raster and JPEG for polygon TEAK_003
Processing polygon TEAK_005...
Clipping with TIFF file: 2023_TEAK_6_323000_4103000_image.tif
Saved clipped raster and JPEG for polygon TEAK_005
Processing polygon TEAK_006...
Clipping with TIFF file: 2023_TEAK_6_316000_4094000_image.tif
Saved clipped raster and JPEG for polygon TEAK_006
Processing polygon TEAK_007...
Clipping with TIFF file: 2023_TEAK_6_317000_4096000_image.tif
Saved clipped raster and JPEG for polygon TEAK_007
Processing polygon TEAK_010...
Clipping with TIFF file: 2023_TEAK_6_317000_4098000_image.tif
Saved clipped raster and JPEG for polygon T

In [ ]:
def plot_trees_on_geotiff_combined(tiff_path, tree_gdf_1, tree_gdf_2, output_path, subfolder):
    with rasterio.open(tiff_path) as src:
        # Read the image data
        image_array = src.read([1, 2, 3])  # Assuming the TIFF has 3 bands (RGB)
        
        # Convert to RGB if necessary
        if image_array.shape[0] == 1:
            image_array = np.stack([image_array[0]] * 3, axis=0)
        elif image_array.shape[0] == 3:
            image_array = np.moveaxis(image_array, 0, -1)

        # Normalize the image to 0-255
        image_array = np.clip(image_array, 0, 255).astype(np.uint8)
        
        # Convert the image array to a PIL image
        img = Image.fromarray(image_array)

        # Create a drawing context
        draw = ImageDraw.Draw(img)
        
        # Plot points from the first GeoDataFrame (tree_gdf_1) as hollow red circles
        for idx, row in tree_gdf_1.iterrows():
            point = row['geometry']
            # Transform point coordinates from CRS to image coordinates
            py, px = src.index(point.x, point.y)
            # Draw the hollow red circle
            draw.ellipse((px - 5, py - 5, px + 5, py + 5), outline='red', width=2)
        
        # Plot points from the second GeoDataFrame (tree_gdf_2) as solid blue circles
        for idx, row in tree_gdf_2.iterrows():
            point = row['geometry']
            print(row['geometry'])
            # Transform point coordinates from CRS to image coordinates
            py, px = src.index(point.x, point.y)
            print(py, px)
            # Draw the solid blue circle
            draw.ellipse((px - 3, py - 3, px + 3, py + 3), fill='blue', outline='blue')
        
        # Define the output file path within the subfolders
        output_filename = os.path.basename(tiff_path).replace('.tif', '_with_trees_combined.png')
        final_output_path = os.path.join(output_path, subfolder)
        os.makedirs(final_output_path, exist_ok=True)

        output_file = os.path.join(final_output_path, output_filename)
        
        # Save the modified image as a PNG file
        img.save(output_file, format='PNG')
        print(f"Saved combined image with trees to {output_file}")

In [ ]:
# Define the CRS for the easting and northing coordinates (e.g., UTM Zone 11N)
crs = 'EPSG:32611'

R_generated_csv = '/data/shared/src/STV/NEON_TEAK/allen/scripts/tree_locations.csv'
app_indv_R_df = pd.read_csv(R_generated_csv)

# Create GeoDataFrame for app_indv_100_df using easting and northing
geometry_1 = [Point(xy) for xy in zip(app_indv_100_locations_df['easting'], app_indv_100_locations_df['northing'])]
tree_gdf_100 = gpd.GeoDataFrame(app_indv_100_locations_df, geometry=geometry_1, crs=crs)

# Create GeoDataFrame for app_indv_400_df using easting and northing
geometry_2 = [Point(xy) for xy in zip(app_indv_400_locations_df['easting'], app_indv_400_locations_df['northing'])]
tree_gdf_400 = gpd.GeoDataFrame(app_indv_400_locations_df, geometry=geometry_2, crs=crs)

print(tree_gdf_100)
print(tree_gdf_400)

tree_gdf_100_2022 = tree_gdf_100[tree_gdf_100['date_x'].str.contains('2022') == True]
tree_gdf_400_2022 = tree_gdf_400[tree_gdf_400['date_x'].str.contains('2022') == True]

app_indv_R_df_filtered = app_indv_R_df.dropna(subset=['adjEasting'])
app_indv_R_df_filtered_2022 = app_indv_R_df_filtered[app_indv_R_df_filtered['date'].str.contains('2022') == True]

geometry_3 = [Point(xy) for xy in zip(app_indv_R_df_filtered['adjEasting'], app_indv_R_df_filtered['adjNorthing'])]
tree_gdf_R = gpd.GeoDataFrame(app_indv_R_df_filtered, geometry=geometry_3, crs=crs)

print(tree_gdf_R)

geometry_4 = [Point(xy) for xy in zip(app_indv_R_df_filtered_2022['adjEasting'], app_indv_R_df_filtered_2022['adjNorthing'])]
tree_gdf_R2 = gpd.GeoDataFrame(app_indv_R_df_filtered_2022, geometry=geometry_4, crs=crs)

                     plotptid          namedLocation      date_x  plotID_x  \
0    TEAK_011.basePlot.vst.33  TEAK_011.basePlot.vst  2022-07-06  TEAK_011   
1    TEAK_011.basePlot.vst.33  TEAK_011.basePlot.vst  2022-07-06  TEAK_011   
2    TEAK_011.basePlot.vst.31  TEAK_011.basePlot.vst  2022-07-06  TEAK_011   
3    TEAK_011.basePlot.vst.33  TEAK_011.basePlot.vst  2022-07-06  TEAK_011   
4    TEAK_011.basePlot.vst.51  TEAK_011.basePlot.vst  2022-07-06  TEAK_011   
..                        ...                    ...         ...       ...   
222  TEAK_007.basePlot.vst.49  TEAK_007.basePlot.vst  2022-07-12  TEAK_007   
223  TEAK_007.basePlot.vst.49  TEAK_007.basePlot.vst  2022-07-12  TEAK_007   
224  TEAK_007.basePlot.vst.49  TEAK_007.basePlot.vst  2022-07-12  TEAK_007   
225  TEAK_007.basePlot.vst.49  TEAK_007.basePlot.vst  2022-07-12  TEAK_007   
226  TEAK_007.basePlot.vst.49  TEAK_007.basePlot.vst  2022-07-12  TEAK_007   

    subplotID             individualID        growthForm       

In [ ]:
# Combine tree_gdf_100 and tree_gdf_400
combined_tree_gdf = gpd.GeoDataFrame(pd.concat([tree_gdf_100, tree_gdf_400], ignore_index=True))
combined_tree_gdf_2022 = gpd.GeoDataFrame(pd.concat([tree_gdf_100_2022, tree_gdf_400_2022], ignore_index=True))

subfolder_with_R = "field(2021-2023)_ortho(2023)_R(2021-2023)_combined"
subfolder_with_R_field_2022_only = "field(2022)_ortho(2023)_R(2022)_combined"

for filename in os.listdir(tif_output_path):
    if filename.endswith('.tif'):
        plot_trees_on_geotiff_combined(os.path.join(tif_output_path, filename), combined_tree_gdf, tree_gdf_R, tree_loc_path, subfolder_with_R)
        plot_trees_on_geotiff_combined(os.path.join(tif_output_path, filename), combined_tree_gdf_2022, tree_gdf_R2, tree_loc_path, subfolder_with_R_field_2022_only)

POINT (321144.474031713 4097134.67552819)
-28885 6891
POINT (321149.0974211 4097119.13729714)
-28730 6937
POINT (321137.963076205 4097125.30134858)
-28792 6826
POINT (321132.797508254 4097132.58626149)
-28864 6774
POINT (321142.657223294 4097129.84509428)
-28837 6873
POINT (321136.322537916 4097137.46801144)
-28913 6810
POINT (321137.319660189 4097133.37812179)
-28872 6820
POINT (321138.656738296 4097130.44869936)
-28843 6833
POINT (321136.09179869 4097136.03719281)
-28899 6807
POINT (321142.996560342 4097123.46677242)
-28773 6876
POINT (321137.96175346 4097108.57065415)
-28624 6826
POINT (321146.817565222 4097113.92722512)
-28678 6915
POINT (321134.618577009 4097110.62878637)
-28645 6793
POINT (321136.970652954 4097106.76475469)
-28606 6816
POINT (321141.070268969 4097113.21740135)
-28671 6857
POINT (321141.177544249 4097099.62111119)
-28535 6858
POINT (321140.605238211 4097112.85773955)
-28667 6853
POINT (321134.968829456 4097112.87806237)
-28667 6796
POINT (321145.621893722 4097099.